<a href="https://colab.research.google.com/github/musa-ib/SpeechScribe/blob/main/speech_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
data_path = '/content/drive/MyDrive/speech_commands_v0.01.tar.gz'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tarfile
with tarfile.open(data_path, 'r:gz') as tar:
  tar.extractall('/content/drive/MyDrive/Speech_commands')
  print('Done')

Done


In [ ]:
pip install librosa

In [ ]:
import librosa
import os
import numpy as np
import json

In [ ]:
dataset_path = '/content/drive/MyDrive/Speech_commands'
json_path = 'data.json'
samples = 22050

def prepare_dataset(dataset_path, json_path, n_mfcc=13, hop_length=512, n_fft=2048):
    data = {
        "mapping": [],
        "labels": [],
        "mfcc": [],
        "files": []
    }

    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        if dirpath is not dataset_path:
          category = dirpath.split('/')[-1]
          data['mapping'].append(category)
          print(f'Processing: {category}')

          for f in filenames:
            file_path = os.path.join(dirpath, f)
            signal, sr = librosa.load(file_path)

            if len(signal) >= samples:
              signal = signal[:samples]

              mfccs = librosa.feature.mfcc(y=signal, n_mfcc=n_mfcc,  hop_length=hop_length, n_fft=n_fft)

              data['labels'].append(i-1)
              data['mfcc'].append(mfccs.T.tolist())
              data['files'].append(file_path)
              print(f'{file_path}: {i-1}')

    with open(json_path, 'w') as fp:
      json.dump(data, fp, indent=4)


In [ ]:
prepare_dataset(dataset_path, json_path)

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/Speech_commands/up/e71b4ce6_nohash_0.wav: 28
/content/drive/MyDrive/Speech_commands/up/471a0925_nohash_0.wav: 28
/content/drive/MyDrive/Speech_commands/up/0ab3b47d_nohash_1.wav: 28
/content/drive/MyDrive/Speech_commands/up/9d32f10a_nohash_1.wav: 28
/content/drive/MyDrive/Speech_commands/up/f2b8fc18_nohash_0.wav: 28
/content/drive/MyDrive/Speech_commands/up/6d2d3b0d_nohash_0.wav: 28
/content/drive/MyDrive/Speech_commands/up/13199b79_nohash_2.wav: 28
/content/drive/MyDrive/Speech_commands/up/5fadb538_nohash_1.wav: 28
/content/drive/MyDrive/Speech_commands/up/1eddce1d_nohash_2.wav: 28
/content/drive/MyDrive/Speech_commands/up/62641b88_nohash_0.wav: 28
/content/drive/MyDrive/Speech_commands/up/1b88bf70_nohash_1.wav: 28
/content/drive/MyDrive/Speech_commands/up/826268f8_nohash_0.wav: 28
/content/drive/MyDrive/Speech_commands/up/b7a0754f_nohash_4.wav: 28
/content/drive/MyDrive/Speech_commands/up/9ff2d2f4_nohash_4.wav: 

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import json

In [ ]:
json_data = '/content/drive/MyDrive/data.json'

In [ ]:
def data_splits(json_path):
  with open(json_path, 'r') as fp:
    data = json.load(fp)
  print(data.keys())

  X = np.array(data['MFCCs'])
  y = np.array(data['labels'])

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)
  X_train = X_train[..., np.newaxis]
  X_val = X_val[..., np.newaxis]
  X_test = X_test[..., np.newaxis]
  return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
# json_data = '/content/drive/MyDrive/data.json'
X_train, X_val, X_test, y_train, y_val, y_test = data_splits(json_data)
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

dict_keys(['mappings', 'labels', 'MFCCs', 'files'])


In [ ]:
model = keras.Sequential()
# layer 1
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=input_shape,kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))

# layer 2
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))

# layer 3
model.add(keras.layers.Conv2D(32, (2, 2), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))


# flatten
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.3))

# output layer
model.add(keras.layers.Dense(30, activation='softmax'))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])






/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 42, 11, 128)         │           1,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 42, 11, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 21, 6, 128)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 19, 4, 64)           │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 19, 4, 64)           │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 10, 2, 64)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 9, 1, 32)            │           8,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 9, 1, 32)            │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 5, 1, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 160)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          10,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           1,950 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 96,446 (376.74 KB)

 Trainable params: 95,998 (374.99 KB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=45)

Epoch 1/45
1475/1475 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.1253 - loss: 3.4423 - val_accuracy: 0.4475 - val_loss: 2.1943
Epoch 2/45
1475/1475 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4120 - loss: 2.1595 - val_accuracy: 0.6443 - val_loss: 1.4353
Epoch 3/45
1475/1475 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5802 - loss: 1.5459 - val_accuracy: 0.7486 - val_loss: 1.0686
Epoch 4/45
1475/1475 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6767 - loss: 1.2153 - val_accuracy: 0.8028 - val_loss: 0.8446
Epoch 5/45
1475/1475 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7363 - loss: 1.0140 - val_accuracy: 0.8215 - val_loss: 0.7542
Epoch 6/45
1475/1475 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7744 - loss: 0.8816 - val_accuracy: 0.8436 - val_loss: 0.6598
Epoch 7/45
1475/1475 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7968 - loss: 0.8009 - val_accuracy: 0.8632 - val_loss: 0.6002
Epoch 8/45
1475/1475 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8156 - loss: 0.7382 

In [ ]:
test_error, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Error: {test_error}, Test Accuracy: {test_accuracy}')

183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9182 - loss: 0.4149
Test Error: 0.3777724504470825, Test Accuracy: 0.918297290802002


In [ ]:
model.save('speech_command.h5')

In [ ]:
model.save('speech_com.keras')

In [ ]:
print("abc")

abc


In [ ]:
import librosa
import tensorflow as tf
import numpy as np

In [ ]:
signal , sr = librosa.load('/content/drive/MyDrive/Speech_commands/yes/0a7c2a8d_nohash_0.wav')
if len(signal) > 22050:
              signal = signal[:22050]
mfccs = librosa.feature.mfcc(y=signal, n_mfcc=13,  hop_length=512, n_fft=2048)
mfccs = mfccs.T.tolist()
mfccs = np.array(mfccs)
input_data = mfccs[np.newaxis,..., np.newaxis]

In [ ]:
model = tf.keras.models.load_model('/content/speech_command.h5')

In [ ]:
input_data.shape

(1, 44, 13, 1)

In [ ]:
prediction = model.predict(input_data)
prediction = np.argmax(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 777ms/step


In [ ]:
prediction

28

In [ ]:
height, width, channels = 44, 13, 1  # Example values
input_shape = (height, width, channels)